### Imports.

In [1]:
import pandas as pd
import numpy as np
import operator
import random
import sqlite3
from z3 import * # must be at least version 4.12.2. 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from time import time
from tqdm import tqdm
from tabulate import tabulate
from itertools import product
from datetime import datetime

pd.set_option("future.no_silent_downcasting", True)

# Class for a "functional" dictionary, which can be also be accessed with round brackets. 
class fdict(dict):
    def __call__(self, k):
        return self[k] 

### Load the frames and perform some further preprocessing for Z3.

In [2]:
# Load the compas csv into a dataframe.
compas = pd.read_csv('data/compas.csv')

# Shuffle the data.
compas = compas.sample(frac=1).reset_index(drop=True)

# Remove rows from compas that have an "Unknown" marital_status.
compas = compas[compas["marital_status"] != "Unknown"]

# Change 'marital_status' column values to 1 and 0, binarily indicating relationship status. 
r_dict = {
    'Single' : 0,   
    'Divorced' : 0,
    'Significant Other' : 1,
    'Married' : 1,
    'Separated' : 0,
    'Widowed' : 0
}
compas["marital_status"] = compas["marital_status"].replace(r_dict)
compas.rename(columns={"marital_status": "partnered"}, inplace=True)
compas["partnered"] = compas["partnered"].astype("int64")

# Change the column values of fta_text, grecid_text, and vrecid_text to numbers. 
for col in ["fta_text", "grecid_text", "vrecid_text"]:
    compas[col] = compas[col].replace({'Low' : 0, 'Medium' : 1, 'High' : 2})
    compas[col] = compas[col].astype("int64")

# Print the column types.
print(compas.dtypes)

male                     int64
age                      int64
juv_fel_count            int64
juv_misd_count           int64
juv_other_count          int64
priors_count             int64
age_at_first_arrest      int64
partnered                int64
slevel                   int64
rs_fta                 float64
rs_grecid              float64
rs_vrecid              float64
ds_fta                   int64
ds_grecid                int64
ds_vrecid                int64
fta_text                 int64
grecid_text              int64
vrecid_text              int64
dtype: object


### Determine the appropriate sorts for the dimensions.

In [3]:
# Initialize the solver.
s = Solver()

# A dictionary mapping a dimension to its Z3 sort.
sorts = {}

# A dictionary mapping a dimension value to its Z3 value.
cast = {}

# Loop over the dimensions and fill the sorts and cast dicts.
for d in compas.columns.values:
    if compas.dtypes[d] == 'float64':
        sorts[d] = RealSort()
        cast[d] = RealVal
    else:
        sorts[d] = IntSort()
        # cast[d] = IntVal # This should work but doesn't due to a bug in Z3.
        cast[d] = lambda x: IntVal(int(x)) # wrap IntVal in a python integer cast to avoid the bug.

# Make a variable fact situation.
x = {d : Const(f'x_{d}', sorts[d]) for d in compas.columns.values}

### Define Z3 constraint formulas, and the manual dimension orders.

In [4]:
# Express case base constraint in terms of logical formulas. 
def eq(X, Y, D):
    return And([X[d] == Y[d] for d in D])

def lteq(X, Y, D, orders):
    return And([orders[d](X[d], Y[d]) for d in D[:-1]])

def lob(X, Y, D, orders):
    return Implies(lteq(Y, X, D, orders), Y[D[-1]] <= X[D[-1]])

def upb(X, Y, D, orders):
    return Implies(lteq(X, Y, D, orders), X[D[-1]] <= Y[D[-1]])

# Define the manual orders.
morders = {
    'male' : operator.le,
    'age' : operator.ge,
    'juv_fel_count' : operator.le,
    'juv_misd_count' : operator.le,
    'juv_other_count' : operator.le,
    'priors_count' : operator.le,
    'age_at_first_arrest' : operator.ge,
    'partnered' : operator.ge,
    'slevel' : operator.le,
    'rs_fta' : operator.le,
    'rs_grecid' : operator.le,
    'rs_vrecid' : operator.le,
    'ds_fta' : operator.le,
    'ds_grecid' : operator.le,
    'ds_vrecid' : operator.le,
    'fta_text' : operator.le,
    'grecid_text' : operator.le,
    'vrecid_text' : operator.le,
}

### Data analyses 1 & 2.

1. First cell computes the main consistency percentages reported in Tables 3 and 4.
2. Second cell finds inconsistent fact situations of the recommended supervision level target dimension, with as input the raw risk scores. An example of this is listed in Table 5.
3. Third cell prints the exact example listed in Table 5.

In [5]:
# Define the set of dimension sets.
TD = compas.columns.values
sD = ['male', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'age_at_first_arrest', 'partnered']
tD = ['slevel', 'rs_fta', 'rs_grecid', 'rs_vrecid', 'ds_fta', 'ds_grecid', 'ds_vrecid', 'fta_text', 'grecid_text', 'vrecid_text']
Ds = [sD + [td] for td in tD]
Ds += [
    ['rs_fta', 'rs_grecid', 'rs_vrecid', 'slevel'],
    ['ds_fta', 'ds_grecid', 'ds_vrecid', 'slevel'],
    ['fta_text', 'grecid_text', 'vrecid_text', 'slevel'],
]

# Initializations.
s = Solver()
ind = ["p_cons", "method"] + list(TD)
methods = ["linreg", "pearson", "manual"]
stats_df = pd.DataFrame(index=ind, dtype=object)
orders = {}

print("Computing stats — this will take a while.")

# Loop over all the sets of dimensions and methods to calculate the statistics.
for D in Ds:
    for m in methods:
        # Initialize stats series.
        stats_series = pd.Series(index=ind, dtype='object')

        # Determine the orders, depending on the chosen method.
        if m == "manual":
            orders = morders
        else:
            # Prepare the data for analysis. 
            data = compas[D].drop_duplicates().to_numpy()
            data = preprocessing.StandardScaler().fit(data).transform(data)
            target = data[:,-1]
            coeffs = {}

            # Determine the coefficients based on the method. 
            if m == "linreg":
                clf = LinearRegression().fit(data, target)
                coeffs = dict(zip(D, clf.coef_))
            elif m == "pearson":
                coeffs = compas.corr()[D[-1]]

            # Determine the dimension orders on the basis of the coefficients.
            for d in D[:-1]:
                orders[d] = operator.le if coeffs[d] > 0 else operator.ge

        # Record orders in stats series. 
        for d in TD:
            if d in D[:-1]:
                if orders[d] == operator.le:
                    stats_series[d] = "+"
                elif orders[d] == operator.ge:
                    stats_series[d] = "-"
            else:
                stats_series[d] = "N/A"

        stats_series[D[-1]] = "target"

        # Drop all the columns from compas that are not in D.
        df = compas[D]

        # Initialize the case base.
        C = [{d : cast[d](r[d]) for d in df.columns} for r in df.iloc]

        # Compute the consistency percentage.
        s.push()
        s.add([f for Y in C for f in [upb(x, Y, D, orders), lob(x, Y, D, orders)]])
        cons = 0
        total = len(C)
        for X in C:
            s.push()
            s.add(eq(x, X, D))
            cons += s.check() == sat
            s.pop()
        s.pop()

        # Write stats to series. 
        stats_series["p_cons"] = round(cons / len(C), 2)
        stats_series["method"] = m

        # Print results and merge with stats df. 
        stats_df = pd.concat([stats_df, stats_series], axis=1)

# Transpose and print the dataframe.
stats_df = stats_df.T.reset_index(drop=True)
print("Done computing stats:")
print(stats_df)

# Save stats_df to the data subdirectory.
stats_df.to_csv("data/stats.csv", index=False)

Computing stats — this will take a while.
Done computing stats:
   p_cons   method male  age juv_fel_count juv_misd_count juv_other_count  \
0     0.0   linreg    -    +             -              -               -   
1    0.02  pearson    +    -             +              +               +   
2    0.02   manual    +    -             +              +               +   
3     0.0   linreg    -    -             -              -               -   
4     0.0  pearson    +    +             +              +               +   
5     0.0   manual    +    +             +              +               +   
6     0.0   linreg    +    -             -              -               -   
7     0.0  pearson    +    -             +              +               +   
8     0.0   manual    +    -             +              +               +   
9     0.0   linreg    +    -             +              +               +   
10    0.0  pearson    +    -             +              +               +   
11    0.0   

In [6]:
# Define string symbols for the less-than and greater-than orders. 
os = {operator.le : "<=", operator.ge : ">="}

# Convert a Z3 number to a python float. 
def conv(x):
    xstr = x.as_string()
    ab = xstr.split("/")
    if len(ab) == 2:
        a = ab[0]
        b = ab[1]
        return float(a) / float(b)
    else: 
        return float(xstr)

# Compares the constraint on a focus fact situation X induced by a precedent case Y. 
def compare(X, Y, orders, D):
    print("Upper bound constraint:")
    for d in D[:-1]:
        print(f"{X[d]} {os[orders[d]]} {Y[d]}: {conv(X[d])} <= {conv(Y[d])}: {orders[d](conv(X[d]), conv(Y[d]))}")
    print("-------------------------------")
    print(f"{X[D[-1]]} <= {Y[D[-1]]}: {conv(X[D[-1]]) <= conv(Y[D[-1]])}")
    print("Lower bound constraint:")
    for d in D[:-1]:
        print(f"{Y[d]} {os[orders[d]]} {X[d]}: {conv(Y[d])} <= {conv(X[d])}: {orders[d](conv(Y[d]), conv(X[d]))}")
    print("-------------------------------")
    print(f"{Y[D[-1]]} <= {X[D[-1]]}: {conv(Y[D[-1]]) <= conv(X[D[-1]])}")
    print()

# Define the set of dimension sets.
Ds = [
    ['rs_fta', 'rs_grecid', 'rs_vrecid', 'slevel'],
    ['ds_fta', 'ds_grecid', 'ds_vrecid', 'slevel'],
    ['fta_text', 'grecid_text', 'vrecid_text', 'slevel'],
]

# Initializations.
s = Solver()
orders = {}

# Loop over all the sets of dimensions and methods to calculate the statistics.
for D in Ds:
    orders = morders

    # Drop all the columns from compas that are not in D.
    df = compas[D]
    df = df.drop_duplicates()

    # Initialize the case base.
    C = [{d : cast[d](r[d]) for d in D} for r in df.iloc]

    # Try to find inconsistencies.
    s.push()
    s.add([f for Y in C for f in [upb(x, Y, D, orders), lob(x, Y, D, orders)]])
    Z = None
    for X in C:
        s.push()
        s.add(eq(x, X, D))
        if s.check() == unsat:
            Z = X
            s.pop()
            break
        s.pop()
    s.pop()

    # Find the precedent causing Z to be inconsistent, if there was any.
    if Z != None:
        s.push()
        s.add(eq(x, Z, D))
        W = None
        for Y in C:
            s.push()
            s.add([upb(x, Y, D, orders), lob(x, Y, D, orders)])
            if s.check() == unsat:
                W = Y
                s.pop()
                break
            s.pop()
        s.pop()
        if W != None:
            print("Found inconsistency:")
            compare(Z, W, orders, D)
        else:
            print("Could not find cause, this should not happen!")
            assert 0 == 1

Found inconsistency:
Upper bound constraint:
15 <= 14: 15.0 <= 14.0: False
-3/100 <= -7/100: -0.03 <= -0.07: False
-173/100 <= -9/5: -1.73 <= -1.8: False
-------------------------------
2 <= 3: True
Lower bound constraint:
14 <= 15: 14.0 <= 15.0: True
-7/100 <= -3/100: -0.07 <= -0.03: True
-9/5 <= -173/100: -1.8 <= -1.73: True
-------------------------------
3 <= 2: False

Found inconsistency:
Upper bound constraint:
0 <= 0: 0.0 <= 0.0: True
1 <= 1: 1.0 <= 1.0: True
1 <= 1: 1.0 <= 1.0: True
-------------------------------
2 <= 1: False
Lower bound constraint:
0 <= 0: 0.0 <= 0.0: True
1 <= 1: 1.0 <= 1.0: True
1 <= 1: 1.0 <= 1.0: True
-------------------------------
1 <= 2: True



In [7]:
focus_rows = (compas["rs_fta"] == 19) & (compas["rs_grecid"] == 0.11) & (compas["rs_vrecid"] == -1.21)
prec_rows = (compas["rs_fta"] == 21) & (compas["rs_grecid"] == 0.14) & (compas["rs_vrecid"] == -0.95)

print("=========================")
print("PREC ROWS")
print("=========================")
print(compas[prec_rows]) 

print("=========================")
print("FOCUS ROWS")
print("=========================")
print(compas[focus_rows])

PREC ROWS
      male  age  juv_fel_count  juv_misd_count  juv_other_count  priors_count  \
7511     1   23              1               0                2             2   

      age_at_first_arrest  partnered  slevel  rs_fta  rs_grecid  rs_vrecid  \
7511                   17          0       3    21.0       0.14      -0.95   

      ds_fta  ds_grecid  ds_vrecid  fta_text  grecid_text  vrecid_text  
7511       3          7          9         0            1            2  
FOCUS ROWS
      male  age  juv_fel_count  juv_misd_count  juv_other_count  priors_count  \
6206     1   31              0               0                0             4   

      age_at_first_arrest  partnered  slevel  rs_fta  rs_grecid  rs_vrecid  \
6206                   28          0       4    19.0       0.11      -1.21   

      ds_fta  ds_grecid  ds_vrecid  fta_text  grecid_text  vrecid_text  
6206       3          8          8         0            2            2  


### Data analysis 3.

First cell prints an example of a fact situation that is inconsistent when we use the raw scores, but consistent when we use the decile scores (assuming the manual dimension orders). This was found using the last code block under data analysis 2. The second cell prints the statistics related to our third data analysis.

In [8]:
# Load the bin frames.
fta_bins = pd.read_csv('data/fta_bins.csv')
grecid_bins = pd.read_csv('data/grecid_bins.csv')
vrecid_bins = pd.read_csv('data/vrecid_bins.csv')

# Map a decile score to a text label as specifed in the practitioners guide. 
def conv(x):
    if 1 <= x <= 4:
        return "Low"
    elif 5 <= x <= 7:
        return "Medium"
    elif 8 <= x <= 10:
        return "High"
    
# Check if the score_text column is equal to conv applied to the decile_score column.
print("Checking that the decile scores are correctly converted to text:")
print((fta_bins["score_text"] == fta_bins["decile_score"].apply(conv)).all())
print((grecid_bins["score_text"] == grecid_bins["decile_score"].apply(conv)).all())
print((vrecid_bins["score_text"] == vrecid_bins["decile_score"].apply(conv)).all())
print()

# Drop the score text columns.
fta_bins = fta_bins[["raw_score", "decile_score"]]
grecid_bins = grecid_bins[["raw_score", "decile_score"]]
vrecid_bins = vrecid_bins[["raw_score", "decile_score"]]

# Rename the columns to match that of the compas df.
fta_bins.rename(columns={"raw_score" : "rs_fta", "decile_score" : "ds_fta"}, inplace=True)
grecid_bins.rename(columns={"raw_score" : "rs_grecid", "decile_score" : "ds_grecid"}, inplace=True)
vrecid_bins.rename(columns={"raw_score" : "rs_vrecid", "decile_score" : "ds_vrecid"}, inplace=True)

# The scale cut-points as specified in the practitioners guide. 
cut_points = {
    "rs_fta" : [2.89, 3.08, 3.24, 3.39, 3.54, 3.69, 3.86, 4.08, 4.38, 8.01],
    "rs_grecid" : [-2.9, -2.5, -2.2, -2, -1.7, -1.5, -1.2, -1, -0.6, 1.9],
    "rs_vrecid" : [-1.3, -0.9, -0.7, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 1.9],
}

# Place input x into a bin according to the cut-points specified by cut_points[c].
def bin_rs(x, c):
    cp = cut_points[c]
    for i, e in enumerate(cp):
        if x <= e:
            return i + 1
    return 10

# Manually bin the raw scores according to the cut-points and compute overlap with ds columns. 
print("Checking consistency with cut-points from practitioner's guide:")
for df in [fta_bins, grecid_bins, vrecid_bins]:
    name = df.columns.values[0]
    scale = name.split('_')[1]
    df[f"ds_{scale}2"] = df[name].apply(lambda x: bin_rs(x, name))
    # Compute the percentage of equal values in ds and ds2 columns.
    good = (df["ds_" + scale] == df["ds_" + scale + "2"]).sum()
    total = len(df)
    print(f"{name}: {good} / {total} = {round(good / total, 2)}")
    df.drop(columns=[f"ds_{scale}2"], inplace=True)
print()

# Group the df's by decile_score, and check for each group what the lowest and highest value of the raw score is.
print("Checking the least and max values for each bin:")
print(fta_bins.groupby("ds_fta").agg({"rs_fta" : ["min", "max"]}).reset_index())
print(grecid_bins.groupby("ds_grecid").agg({"rs_grecid" : ["min", "max"]}).reset_index())
print(vrecid_bins.groupby("ds_vrecid").agg({"rs_vrecid" : ["min", "max"]}).reset_index())

Checking that the decile scores are correctly converted to text:
True
True
True

Checking consistency with cut-points from practitioner's guide:
rs_fta: 212 / 12526 = 0.02
rs_grecid: 742 / 12510 = 0.06
rs_vrecid: 3446 / 12521 = 0.28

Checking the least and max values for each bin:
  ds_fta rs_fta      
            min   max
0      1   11.0  16.0
1      2   16.0  19.0
2      3   18.0  21.0
3      4   20.0  23.0
4      5   22.0  25.0
5      6   23.0  27.0
6      7   25.0  29.0
7      8   27.0  31.0
8      9   29.0  35.0
9     10   34.0  50.0
  ds_grecid rs_grecid      
                  min   max
0         1     -3.21 -1.39
1         2     -1.65 -0.92
2         3     -1.30 -0.60
3         4     -1.02 -0.39
4         5     -0.81 -0.19
5         6     -0.58  0.01
6         7     -0.36  0.19
7         8     -0.13  0.39
8         9      0.13  0.67
9        10      0.44  2.36
  ds_vrecid rs_vrecid      
                  min   max
0         1     -4.63 -2.95
1         2     -2.94 -2.56
2     

In [9]:
# Initializations.
s = Solver()
ind = ["name", "size", "p_cons"]
stats_df = pd.DataFrame(index=ind, dtype='object')
orders = {}

# Loop over all the sets of dimensions and methods to calculate the statistics.
for df in [fta_bins, grecid_bins, vrecid_bins]:
    D = df.columns.values

    # Record some stats.
    stats_series = pd.Series(index=ind, dtype='object')
    stats_series["size"] = len(df)
    stats_series["name"] = D[0]

    # Set dimension orders to the manual choice. 
    orders = morders

    # Initialize the case base.
    C = [{d : cast[d](r[d]) for d in df.columns} for r in df.iloc]

    # Compute the consistency percentage.
    s.push()
    s.add([f for Y in C for f in [upb(x, Y, D, orders), lob(x, Y, D, orders)]])
    cons = 0
    total = len(df)
    for X in C:
        s.push()
        s.add(eq(x, X, D))
        cons += s.check() == sat
        s.pop()
    s.pop()

    # Write stats to series. 
    stats_series["p_cons"] = round(cons / len(C), 2)

    # Print results and merge with stats df. 
    stats_df = pd.concat([stats_df, stats_series], axis=1)

# Transpose and print the dataframe.
stats_df = stats_df.T.reset_index(drop=True)
print(stats_df)

        name   size p_cons
0     rs_fta  12526   0.47
1  rs_grecid  12510   0.23
2  rs_vrecid  12521    1.0
